<font size="5" color="#2874a6">Reinforcement Learning with Q-learning</font>

In [1]:
%reset -f

## Environment

In [2]:
# Base Environment class
class Environment:
    def __init__(self, shape, landmarks: list = None):
        self.shape = shape
        self.landmarks = landmarks
        self.reset()
    
    def reset(self):
        """Defines and resets the entire state space."""
        self.state = None
        return self.state

    def step(self, action):
        """Defines the next state and reward regarding to an action at current state."""
        reward, done = None, False
        return self.state, reward, done

In [3]:
from typing import Literal, get_args

# Movement types
_MOVETYPES = Literal["up", "down", "left", "right"]

# GridEnvironment class
class GridEnvironment(Environment):
    def __init__(self, shape=(4, 4), landmarks=[(0, 3), (1, 1)]):
        super().__init__(shape, landmarks)
    
    def reset(self):
        self.state = (3, 0)
        return self.state
    
    def give_reward(self):
        if self.state == self.landmarks[0]:
            return 1
        elif self.state == self.landmarks[1]:
            return -1 # penalty
        else:
            return 0
    
    def is_terminal(self):
        return self.state == self.landmarks[0]
    
    def step(self, action: _MOVETYPES):
        next_state = list(self.state)
        match(action):
            case "up":
                next_state[0] = max(0, self.state[0] - 1)
            case "down":
                next_state[0] = min(self.shape[0] - 1, self.state[0] + 1)
            case "left":
                next_state[1] = max(0, self.state[1] - 1)
            case "right":
                next_state[1] = min(self.shape[1] - 1, self.state[1] + 1)
        self.state = tuple(next_state)

        reward = self.give_reward()
        done = self.is_terminal()

        return self.state, reward, done

## Agent

The Q-learning algorithm uses the following formula to update the Q-value for a state-action-pair:

$Q(s,a) = Q(s,a) + \alpha \cdot [R + \gamma \cdot \underset{a} \max\ Q(s',a') - Q(s,a)]$

with:\
$Q(s,a)$ : Q-value for state $s$ and action $a$\
$\alpha$ : learning rate\
$R$ : immediate reward for taking action $a$ in state $s$\
$\gamma$ : discount factor, representing the importance of future rewards\
$\max Q(s',a')$ : maximum Q-value for the the next state $s'$, representing the best possible reward achievable from that state.

In [4]:
import numpy as np
import random

# Base Agent class
class Agent:
    pass

# QLearning Agent class
class QLearningAgent(Agent):
    def __init__(self, environment: Environment, actions: list = [], alpha=0.1, gamma=0.9, epsilon=0.1):
        self.environment = environment
        self.actions = list(actions)
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        # Q-table
        # A Q-value, denoted with Q(s,a), represents the expected cumulative reward for choosing action a in state s
        self.q_table = np.zeros(self.environment.shape + (len(self.actions),))

    def best_action(self, state):
        return self.actions[np.argmax(self.q_table[state])]
    
    # The agent uses an epsilon-greedy strategy
    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return self.actions[np.random.randint(len(self.actions))] # explore
        else:
            return self.best_action(state) # exploit
    
    def update_q_value(self, state, action, reward, next_state):
        a = self.actions.index(action)
        best_next_q = np.max(self.q_table[next_state])
        self.q_table[state][a] += self.alpha * (reward + self.gamma * best_next_q - self.q_table[state][a])

    def fit(self, episodes=1000, verbose=False):
        for episode in range(episodes):
            state = self.environment.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.choose_action(state)
                next_state, reward, done = self.environment.step(action)
                self.update_q_value(state, action, reward, next_state)
                state = next_state
                total_reward += reward

            if verbose:
                print(f"Episode: {episode}, Total reward: {total_reward}")

    def path(self, state=None):
        if state is None:
            state = self.environment.reset()
        else:
            self.environment.state = state

        states = [state]
        actions = []
        done = False
        while not done:
            action = self.best_action(state)
            state, reward, done = self.environment.step(action)
            states.append(state)
            actions.append(action)
            if done:
                break
        return states, actions

## Train the Agent

In [5]:
env = GridEnvironment()
agent = QLearningAgent(environment=env, actions=get_args(_MOVETYPES))
agent.fit()

## Use Agent for Optimal Path

In [10]:
states, actions = agent.path(state=(2, 1))
print("Optimal path to goal:", states)
print("Optimale moves from state to goal:", actions)

Optimal path to goal: [(2, 1), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]
Optimale moves from state to goal: ['left', 'up', 'up', 'right', 'right', 'right']
